<a href="https://colab.research.google.com/github/AndresPautrat/Concurrente/blob/main/Go_philosofos_forks_wr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install golang

In [ ]:
%%writefile dp.go
package main

import (
    //"fmt"
    "sync"
)

func philosopher(id int, rightFork, leftFork,sync.Mutex){
    for{
        //thinking
        rightFork.Lock()
        leftFork.Lock()
        //eat
        rightFork.Unlock()
        leftFork.Unlock()
    }
}

func main(){
    n:=5
    fork :=make([]sync.Mutex,n)
    for i:=0;i<n;i++{
        go philosopher(i,fork[i],forks[i+1])
    }
    philosopher(n-1,forks[n-1],forks[0])
}

Overwriting dp.go


In [ ]:
!go run dp.go

# command-line-arguments
./dp.go:8:17: syntax error: mixed named and unnamed function parameters


In [ ]:
!apt install spin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  spin
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 559 kB of archives.
After this operation, 1,776 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 spin amd64 6.4.6+dfsg-2 [559 kB]
Fetched 559 kB in 1s (530 kB/s)
Selecting previously unselected package spin.
(Reading database ... 153218 files and directories currently installed.)
Preparing to unpack .../spin_6.4.6+dfsg-2_amd64.deb ...
Unpacking spin (6.4.6+dfsg-2) ...
Setting up spin (6.4.6+dfsg-2) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [ ]:
%%writefile dp1.pml
#define wait(s) atomic{s>0->s--}
#define signal(s) s++

byte forks[5]={1,1,1,1,1}

active[5] proctype P(){
    do 
    ::
      //think
      wait(forks[_pid])
      wait(forks[(_pid+1)%5])
      //eat
      signal(forks[_pid])
      signal(forks[(_pid+1)%5])
    od
}

Overwriting dp1.pml


In [ ]:
!spin -run dp1.pml

pan:1: invalid end state (at depth 56)
pan: wrote dp1.pml.trail

(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 52 byte, depth reached 57, errors: 1
       58 states, stored
       16 states, matched
       74 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.004	equivalent memory usage for states (stored*(State-vector + overhead))
    0.291	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.730	total actual memory usage



pan: elapsed time 0 seconds


In [ ]:
%%writefile dp2.pml
#define wait(s) atomic{s>0->s--}
#define signal(s) s++

byte forks[5]={1,1,1,1,1}
int room=4

active[5] proctype P(){
    do 
    ::
      //think
      wait(room)
      wait(forks[_pid])
      wait(forks[(_pid+1)%5])
      //eat
      signal(forks[_pid])
      signal(forks[(_pid+1)%5])
      signal(room)
    od
}

Overwriting dp2.pml


In [ ]:
!spin -run dp2.pml


(Spin Version 6.4.6 -- 2 December 2016)
	+ Partial Order Reduction

Full statespace search for:
	never claim         	- (none specified)
	assertion violations	+
	cycle checks       	- (disabled by -DSAFETY)
	invalid end states	+

State-vector 52 byte, depth reached 2305, errors: 0
     3111 states, stored
     9280 states, matched
    12391 transitions (= stored+matched)
        0 atomic steps
hash conflicts:         0 (resolved)

Stats on memory usage (in Megabytes):
    0.237	equivalent memory usage for states (stored*(State-vector + overhead))
    0.389	actual memory usage for states
  128.000	memory used for hash table (-w24)
    0.534	memory used for DFS stack (-m10000)
  128.827	total actual memory usage


unreached in proctype P
	dp2.pml:19, state 16, "-end-"
	(1 of 16 states)

pan: elapsed time 0 seconds


In [ ]:
%%writefile problema1.pml

#define waitW(w,r) atomic{w>0 -> r>0 -> w--}
#define waitR(w,r) atomic{w>0 -> r--}
#define signal(s) {s++}

byte writterOnRoom = 1
byte readersOnRoom = 1

active[5] proctype W(){
    waitW(writterOnRoom, readersOnRoom)
    printf("in writter: %d -- %d -- %d\n", _pid, writterOnRoom, readersOnRoom)
    //crit
    signal(writterOnRoom)
    printf("out writter: %d\n", _pid)
}

active[5] proctype R(){
   waitR(writterOnRoom, readersOnRoom)
   printf("\t in reader: %d -- %d -- %d\n", _pid, writterOnRoom, readersOnRoom)
   //crit
   signal(readersOnRoom)
   printf("\t out reader: %d\n", _pid)
}

Overwriting problema1.pml


In [ ]:
!spin problema1.pml

spin: problema1.pml:18, Error: value (-1->255 (8)) truncated in assignment
                                  	 in reader: 7 -- 0 -- 254
                          	 in reader: 5 -- 0 -- 254
      in writter: 0 -- 0 -- 254
                                          	 in reader: 9 -- 0 -- 255
                          	 out reader: 5
spin: problema1.pml:21, Error: value (256->0 (8)) truncated in assignment
                                          	 out reader: 9
      out writter: 0
spin: problema1.pml:18, Error: value (-1->255 (8)) truncated in assignment
          in writter: 1 -- 0 -- 255
spin: problema1.pml:10, Error: value (-1->255 (8)) truncated in assignment
spin: problema1.pml:13, Error: value (256->0 (8)) truncated in assignment
          out writter: 1
spin: problema1.pml:21, Error: value (256->0 (8)) truncated in assignment
                                      	 in reader: 8 -- 0 -- 0
                                  	 out reader: 7
                                      	 out

In [ ]:
%%writefile smokers.pml

active agent